# Risk Modelling & Portfolio Optimization in Python
**Week 5 - Financial ML Bootcamp (INSTRUCTOR VERSION)**

## Overview
This instructor notebook demonstrates risk modelling and portfolio optimization techniques with complete solutions and teaching notes.

**Teaching Objectives:**
- Guide students through Modern Portfolio Theory implementation
- Explain Monte Carlo simulation intuition
- Demonstrate VaR/CVaR practical applications
- Show efficient frontier construction and interpretation

**Instructor Notes:**
- Emphasize diversification benefits through correlation analysis
- Use visualizations to explain risk-return trade-offs
- Connect theory to practical portfolio management
- Highlight assumptions and limitations of MPT

---

In [ ]:
# Parameters and Configuration
SEED = 42
SAMPLE_MODE = True  # Set to False for full analysis
DATA_PATH = "data/portfolio_assets.csv"

# Portfolio configuration
ASSETS = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']
START_DATE = '2019-01-01'
END_DATE = '2024-01-01'
NUM_PORTFOLIOS = 10000 if not SAMPLE_MODE else 1000
CONFIDENCE_LEVEL = 0.95
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate

print(f"Configuration:")
print(f"- Sample Mode: {SAMPLE_MODE}")
print(f"- Assets: {ASSETS}")
print(f"- Date Range: {START_DATE} to {END_DATE}")
print(f"- Number of Portfolios: {NUM_PORTFOLIOS}")
print(f"- Confidence Level: {CONFIDENCE_LEVEL}")

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(SEED)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Step 1: Data Loading and Preprocessing

**Teaching Notes:**
- Explain why we use adjusted closing prices (accounts for dividends/splits)
- Discuss the importance of sufficient historical data for reliable statistics
- Highlight the trade-off between data length and relevance (older data may not reflect current market conditions)

In [ ]:
def load_portfolio_data(assets, start_date, end_date, sample_mode=True):
    """
    Load historical price data for portfolio assets.
    
    INSTRUCTOR NOTE: This function includes robust error handling and fallback synthetic data
    generation to ensure the notebook runs in all environments.
    """
    try:
        print(f"📊 Loading data for {len(assets)} assets...")
        
        # Download data from Yahoo Finance
        data = yf.download(assets, start=start_date, end=end_date, progress=False)
        prices_df = data['Adj Close'].dropna()
        
        # Ensure we have data for all assets
        if prices_df.empty or len(prices_df.columns) < len(assets):
            raise ValueError("Incomplete data downloaded")
            
        print(f"✅ Successfully loaded {len(prices_df)} days of data")
        
    except Exception as e:
        print(f"⚠️ Data download failed: {e}")
        if sample_mode:
            print("🔄 Generating synthetic data for demonstration...")
            prices_df = generate_synthetic_data(assets, start_date, end_date)
        else:
            raise
    
    # Calculate daily returns
    returns_df = prices_df.pct_change().dropna()
    
    # Display basic statistics
    print(f"\nData Summary:")
    print(f"- Date range: {prices_df.index[0].strftime('%Y-%m-%d')} to {prices_df.index[-1].strftime('%Y-%m-%d')}")
    print(f"- Number of trading days: {len(returns_df)}")
    print(f"- Assets: {list(prices_df.columns)}")
    
    return prices_df, returns_df

def generate_synthetic_data(assets, start_date, end_date):
    """
    Generate synthetic price data for demonstration purposes.
    
    INSTRUCTOR NOTE: This creates realistic synthetic data with:
    - Reasonable return/volatility parameters for tech stocks
    - Positive correlations typical of same-sector assets
    - Geometric Brownian motion price paths
    """
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    n_days = len(dates)
    
    # Synthetic parameters (realistic for tech stocks)
    np.random.seed(SEED)
    initial_prices = np.random.uniform(100, 300, len(assets))
    annual_returns = np.random.uniform(0.05, 0.25, len(assets))
    volatilities = np.random.uniform(0.15, 0.35, len(assets))
    
    # Generate correlated returns (tech stocks tend to be positively correlated)
    base_correlation = 0.5
    correlation_matrix = np.full((len(assets), len(assets)), base_correlation)
    np.fill_diagonal(correlation_matrix, 1.0)
    
    # Add some randomness to correlations
    for i in range(len(assets)):
        for j in range(i+1, len(assets)):
            correlation_matrix[i,j] = correlation_matrix[j,i] = np.random.uniform(0.3, 0.7)
    
    # Generate returns using multivariate normal distribution
    daily_returns = np.random.multivariate_normal(
        annual_returns / 252,  # Convert to daily
        np.outer(volatilities, volatilities) * correlation_matrix / 252,
        n_days
    )
    
    # Convert to prices using geometric Brownian motion
    prices = np.zeros((n_days, len(assets)))
    prices[0] = initial_prices
    
    for i in range(1, n_days):
        prices[i] = prices[i-1] * (1 + daily_returns[i])
    
    return pd.DataFrame(prices, index=dates, columns=assets)

# Load the data
prices_df, returns_df = load_portfolio_data(ASSETS, START_DATE, END_DATE, SAMPLE_MODE)

# Display first few rows
print(f"\nReturns Data (first 5 rows):")
print(returns_df.head())

## Step 2: Calculate Portfolio Statistics

**Teaching Notes:**
- Emphasize the difference between sample statistics and population parameters
- Explain why we annualize returns and covariance (252 trading days convention)
- Discuss the importance of correlation in portfolio construction
- Point out that correlations can change over time (not constant as MPT assumes)

In [ ]:
def calculate_portfolio_stats(returns_df):
    """
    Calculate portfolio statistics from returns data.
    
    INSTRUCTOR NOTE: We annualize using 252 trading days, which is the market standard.
    Alternative approaches include 365 days or actual trading days in the period.
    """
    # Annualize returns and covariance (252 trading days per year)
    expected_returns = returns_df.mean() * 252
    cov_matrix = returns_df.cov() * 252
    correlation_matrix = returns_df.corr()
    
    return expected_returns, cov_matrix, correlation_matrix

def display_portfolio_stats(expected_returns, cov_matrix, correlation_matrix):
    """Display portfolio statistics in a formatted way."""
    print("📈 Portfolio Statistics Summary")
    print("=" * 50)
    
    # Expected returns
    print("\n🎯 Expected Annual Returns:")
    for asset, ret in expected_returns.items():
        print(f"  {asset}: {ret:.2%}")
    
    # Volatilities (standard deviations)
    print("\n📊 Annual Volatilities:")
    volatilities = np.sqrt(np.diag(cov_matrix))
    for asset, vol in zip(expected_returns.index, volatilities):
        print(f"  {asset}: {vol:.2%}")
    
    # Sharpe ratios (individual assets)
    print(f"\n⚡ Individual Sharpe Ratios (Risk-free rate: {RISK_FREE_RATE:.1%}):")
    for asset, ret, vol in zip(expected_returns.index, expected_returns, volatilities):
        sharpe = (ret - RISK_FREE_RATE) / vol
        print(f"  {asset}: {sharpe:.3f}")
    
    print(f"\n🔗 Correlation Matrix:")
    print(correlation_matrix.round(3))

# Calculate portfolio statistics
expected_returns, cov_matrix, correlation_matrix = calculate_portfolio_stats(returns_df)
display_portfolio_stats(expected_returns, cov_matrix, correlation_matrix)

# INSTRUCTOR ONLY: Additional analysis
print("\n" + "="*60)
print("🎓 INSTRUCTOR ANALYSIS: Correlation Insights")
print("="*60)

# Analyze correlation structure
corr_values = correlation_matrix.values
upper_triangle = corr_values[np.triu_indices_from(corr_values, k=1)]
avg_correlation = np.mean(upper_triangle)
min_correlation = np.min(upper_triangle)
max_correlation = np.max(upper_triangle)

print(f"Average pairwise correlation: {avg_correlation:.3f}")
print(f"Range: {min_correlation:.3f} to {max_correlation:.3f}")
print(f"Interpretation: {'High' if avg_correlation > 0.7 else 'Moderate' if avg_correlation > 0.4 else 'Low'} correlation among assets")

# Diversification potential
print(f"\nDiversification Potential:")
if avg_correlation > 0.7:
    print("⚠️ High correlation - limited diversification benefits")
elif avg_correlation > 0.4:
    print("✅ Moderate correlation - good diversification potential")
else:
    print("🎯 Low correlation - excellent diversification potential")

## Step 3: Monte Carlo Portfolio Simulation

**Teaching Notes:**
- Explain why we use random weights that sum to 1 (budget constraint)
- Discuss the efficiency of Monte Carlo vs analytical methods
- Emphasize that this explores the full risk-return space, not just the efficient frontier
- Point out that most portfolios will be sub-optimal (inefficient)

In [ ]:
def generate_random_portfolios(expected_returns, cov_matrix, num_portfolios, risk_free_rate):
    """
    Generate random portfolios using Monte Carlo simulation.
    
    INSTRUCTOR NOTE: This function demonstrates the Monte Carlo approach to portfolio
    optimization. We generate random weights using uniform distribution, but other
    approaches (e.g., Dirichlet distribution) could be used for different weight distributions.
    """
    n_assets = len(expected_returns)
    results = {
        'returns': [],
        'volatility': [],
        'sharpe_ratio': [],
        'weights': []
    }
    
    print(f"🎲 Generating {num_portfolios:,} random portfolios...")
    
    # INSTRUCTOR NOTE: Using vectorized operations for efficiency
    # Generate all random weights at once
    all_weights = np.random.random((num_portfolios, n_assets))
    all_weights = all_weights / np.sum(all_weights, axis=1, keepdims=True)
    
    for i in range(num_portfolios):
        weights = all_weights[i]
        
        # Calculate portfolio return
        portfolio_return = np.sum(expected_returns * weights)
        
        # Calculate portfolio volatility using matrix multiplication
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        
        # Calculate Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        
        # Store results
        results['returns'].append(portfolio_return)
        results['volatility'].append(portfolio_volatility)
        results['sharpe_ratio'].append(sharpe_ratio)
        results['weights'].append(weights)
        
        # Progress indicator
        if (i + 1) % (num_portfolios // 10) == 0:
            print(f"  Progress: {((i + 1) / num_portfolios) * 100:.0f}%")
    
    # Convert to numpy arrays
    for key in ['returns', 'volatility', 'sharpe_ratio']:
        results[key] = np.array(results[key])
    
    print(f"✅ Generated {num_portfolios:,} portfolios")
    
    return results

# Generate random portfolios
portfolio_results = generate_random_portfolios(
    expected_returns, cov_matrix, NUM_PORTFOLIOS, RISK_FREE_RATE
)

# Display summary statistics
print(f"\n📊 Portfolio Simulation Results:")
print(f"Returns - Min: {portfolio_results['returns'].min():.2%}, Max: {portfolio_results['returns'].max():.2%}")
print(f"Volatility - Min: {portfolio_results['volatility'].min():.2%}, Max: {portfolio_results['volatility'].max():.2%}")
print(f"Sharpe Ratio - Min: {portfolio_results['sharpe_ratio'].min():.3f}, Max: {portfolio_results['sharpe_ratio'].max():.3f}")

# INSTRUCTOR ONLY: Portfolio efficiency analysis
print("\n" + "="*60)
print("🎓 INSTRUCTOR ANALYSIS: Portfolio Efficiency")
print("="*60)

efficient_portfolios = portfolio_results['sharpe_ratio'] > 1.0
pct_efficient = np.mean(efficient_portfolios) * 100
print(f"Portfolios with Sharpe > 1.0: {pct_efficient:.1f}%")
print(f"This shows that most random portfolios are sub-optimal")
print(f"Efficient portfolios are rare - highlighting the value of optimization")

## Step 4: Identify Optimal Portfolio + Analytical Optimization

**Teaching Notes:**
- Compare Monte Carlo results with analytical optimization
- Explain the difference between numerical and analytical solutions
- Discuss when each approach is preferred
- Highlight the efficiency gains from analytical methods

In [ ]:
# INSTRUCTOR ONLY: Analytical Portfolio Optimization
def analytical_portfolio_optimization(expected_returns, cov_matrix, risk_free_rate):
    """
    Solve portfolio optimization analytically using scipy.optimize.
    
    INSTRUCTOR NOTE: This demonstrates the analytical approach to portfolio optimization,
    which is more efficient and precise than Monte Carlo simulation.
    """
    n_assets = len(expected_returns)
    
    # Objective function: minimize negative Sharpe ratio
    def negative_sharpe_ratio(weights):
        portfolio_return = np.sum(expected_returns * weights)
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        return -(portfolio_return - risk_free_rate) / portfolio_volatility
    
    # Constraints: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Bounds: no short selling (weights >= 0)
    bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Initial guess: equal weights
    initial_guess = np.array([1/n_assets] * n_assets)
    
    # Optimize
    result = minimize(negative_sharpe_ratio, initial_guess, 
                     method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        optimal_weights = result.x
        portfolio_return = np.sum(expected_returns * optimal_weights)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        
        return {
            'weights': optimal_weights,
            'return': portfolio_return,
            'volatility': portfolio_volatility,
            'sharpe_ratio': sharpe_ratio,
            'optimization_success': True
        }
    else:
        return {'optimization_success': False}

def find_optimal_portfolio(portfolio_results, expected_returns, assets):
    """Find the portfolio with maximum Sharpe ratio from Monte Carlo results."""
    # Find index of maximum Sharpe ratio
    max_sharpe_idx = np.argmax(portfolio_results['sharpe_ratio'])
    
    # Extract optimal portfolio characteristics
    optimal_portfolio = {
        'weights': portfolio_results['weights'][max_sharpe_idx],
        'return': portfolio_results['returns'][max_sharpe_idx],
        'volatility': portfolio_results['volatility'][max_sharpe_idx],
        'sharpe_ratio': portfolio_results['sharpe_ratio'][max_sharpe_idx]
    }
    
    return optimal_portfolio, max_sharpe_idx

def display_optimal_portfolio(optimal_portfolio, assets, method_name="Monte Carlo"):
    """Display optimal portfolio characteristics."""
    print(f"🏆 Optimal Portfolio - {method_name} Method")
    print("=" * 50)
    print(f"Expected Return: {optimal_portfolio['return']:.2%}")
    print(f"Volatility: {optimal_portfolio['volatility']:.2%}")
    print(f"Sharpe Ratio: {optimal_portfolio['sharpe_ratio']:.3f}")
    
    print(f"\n💼 Portfolio Weights:")
    weights_df = pd.DataFrame({
        'Asset': assets,
        'Weight': optimal_portfolio['weights'],
        'Weight %': optimal_portfolio['weights'] * 100
    })
    weights_df = weights_df.sort_values('Weight %', ascending=False)
    
    for _, row in weights_df.iterrows():
        print(f"  {row['Asset']}: {row['Weight %']:.1f}%")
    
    return weights_df

# Find optimal portfolio using Monte Carlo
mc_optimal_portfolio, max_sharpe_idx = find_optimal_portfolio(portfolio_results, expected_returns, ASSETS)
mc_weights_df = display_optimal_portfolio(mc_optimal_portfolio, ASSETS, "Monte Carlo")

# INSTRUCTOR ONLY: Compare with analytical optimization
print("\n" + "="*60)
print("🎓 INSTRUCTOR COMPARISON: Analytical vs Monte Carlo")
print("="*60)

analytical_result = analytical_portfolio_optimization(expected_returns, cov_matrix, RISK_FREE_RATE)

if analytical_result['optimization_success']:
    analytical_weights_df = display_optimal_portfolio(analytical_result, ASSETS, "Analytical")
    
    # Compare results
    print(f"\n📊 Method Comparison:")
    print(f"Monte Carlo Sharpe Ratio: {mc_optimal_portfolio['sharpe_ratio']:.6f}")
    print(f"Analytical Sharpe Ratio: {analytical_result['sharpe_ratio']:.6f}")
    difference = analytical_result['sharpe_ratio'] - mc_optimal_portfolio['sharpe_ratio']
    print(f"Difference: {difference:.6f} ({difference/mc_optimal_portfolio['sharpe_ratio']*100:+.2f}%)")
    
    # Weight comparison
    print(f"\n💼 Weight Comparison:")
    comparison_df = pd.DataFrame({
        'Asset': ASSETS,
        'Monte Carlo': mc_optimal_portfolio['weights'] * 100,
        'Analytical': analytical_result['weights'] * 100,
        'Difference': (analytical_result['weights'] - mc_optimal_portfolio['weights']) * 100
    })
    comparison_df = comparison_df.round(2)
    print(comparison_df.to_string(index=False))
    
    print(f"\n🎯 Teaching Point:")
    print(f"Analytical optimization typically provides better results due to:")
    print(f"  - Exact mathematical solution vs sampling approximation")
    print(f"  - No dependence on random sampling")
    print(f"  - Guaranteed global optimum for convex problems")
else:
    print("⚠️ Analytical optimization failed")

# Use analytical result if available, otherwise Monte Carlo
optimal_portfolio = analytical_result if analytical_result.get('optimization_success') else mc_optimal_portfolio
weights_df = analytical_weights_df if analytical_result.get('optimization_success') else mc_weights_df

## Step 5: Advanced Visualization and Analysis

**Teaching Notes:**
- Use multiple visualization types to reinforce concepts
- Explain the shape and curvature of the efficient frontier
- Connect visual patterns to underlying mathematical relationships
- Emphasize practical interpretation of results

In [ ]:
def plot_comprehensive_analysis(portfolio_results, optimal_portfolio, assets, 
                               expected_returns, cov_matrix, correlation_matrix):
    """
    Create comprehensive visualizations for portfolio analysis.
    
    INSTRUCTOR NOTE: This enhanced visualization function provides multiple perspectives
    on the portfolio optimization problem, helping students develop intuition.
    """
    fig = plt.figure(figsize=(20, 15))
    
    # Create a complex subplot layout
    gs = fig.add_gridspec(3, 3, height_ratios=[2, 1, 1], width_ratios=[2, 1, 1])
    
    # 1. Main Efficient Frontier (large plot)
    ax_main = fig.add_subplot(gs[0, 0])
    
    # Color by Sharpe ratio
    scatter = ax_main.scatter(portfolio_results['volatility'], portfolio_results['returns'], 
                             c=portfolio_results['sharpe_ratio'], cmap='viridis', 
                             alpha=0.6, s=20, edgecolors='none')
    
    # Highlight optimal portfolio
    ax_main.scatter(optimal_portfolio['volatility'], optimal_portfolio['return'], 
                   marker='*', color='red', s=500, label='Optimal Portfolio',
                   edgecolors='darkred', linewidth=2)
    
    # Plot individual assets
    asset_volatilities = [np.sqrt(cov_matrix.loc[asset, asset]) for asset in assets]
    ax_main.scatter(asset_volatilities, expected_returns, marker='D', s=150, 
                   color='orange', alpha=0.9, label='Individual Assets',
                   edgecolors='darkorange', linewidth=2)
    
    # Add asset labels with better positioning
    for i, asset in enumerate(assets):
        ax_main.annotate(asset, (asset_volatilities[i], expected_returns[i]), 
                        xytext=(10, 10), textcoords='offset points', 
                        fontsize=10, fontweight='bold',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    ax_main.set_xlabel('Portfolio Volatility (Risk)', fontsize=12)
    ax_main.set_ylabel('Expected Return', fontsize=12)
    ax_main.set_title('Efficient Frontier Analysis\nRisk vs Return Trade-off', fontsize=14, fontweight='bold')
    ax_main.legend(fontsize=10)
    ax_main.grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax_main)
    cbar.set_label('Sharpe Ratio', fontsize=10)
    
    # 2. Sharpe Ratio Distribution
    ax_sharpe = fig.add_subplot(gs[0, 1])
    ax_sharpe.hist(portfolio_results['sharpe_ratio'], bins=50, alpha=0.7, 
                  color='skyblue', edgecolor='black', density=True)
    ax_sharpe.axvline(optimal_portfolio['sharpe_ratio'], color='red', linestyle='--', 
                     linewidth=2, label=f'Optimal: {optimal_portfolio["sharpe_ratio"]:.3f}')
    ax_sharpe.set_xlabel('Sharpe Ratio')
    ax_sharpe.set_ylabel('Density')
    ax_sharpe.set_title('Sharpe Ratio\nDistribution')
    ax_sharpe.legend()
    ax_sharpe.grid(True, alpha=0.3)
    
    # 3. Portfolio Weights (Optimal)
    ax_weights = fig.add_subplot(gs[0, 2])
    colors = plt.cm.Set3(np.linspace(0, 1, len(assets)))
    wedges, texts, autotexts = ax_weights.pie(optimal_portfolio['weights'], 
                                             labels=assets, autopct='%1.1f%%', 
                                             colors=colors, startangle=90)
    ax_weights.set_title('Optimal Portfolio\nWeights')
    
    # 4. Correlation Heatmap
    ax_corr = fig.add_subplot(gs[1, 0])
    sns.heatmap(correlation_matrix, annot=True, cmap='RdYlBu_r', center=0,
                square=True, ax=ax_corr, cbar_kws={'shrink': 0.8})
    ax_corr.set_title('Asset Correlation Matrix')
    
    # 5. Risk-Return by Asset
    ax_risk_return = fig.add_subplot(gs[1, 1])
    volatilities = [np.sqrt(cov_matrix.loc[asset, asset]) for asset in assets]
    bars = ax_risk_return.bar(range(len(assets)), expected_returns, 
                             alpha=0.7, color='lightgreen', label='Expected Return')
    ax_risk_return2 = ax_risk_return.twinx()
    bars2 = ax_risk_return2.bar([i+0.4 for i in range(len(assets))], volatilities, 
                               alpha=0.7, color='lightcoral', width=0.4, label='Volatility')
    
    ax_risk_return.set_xlabel('Assets')
    ax_risk_return.set_ylabel('Expected Return', color='green')
    ax_risk_return2.set_ylabel('Volatility', color='red')
    ax_risk_return.set_xticks(range(len(assets)))
    ax_risk_return.set_xticklabels(assets, rotation=45)
    ax_risk_return.set_title('Individual Asset\nRisk-Return Profile')
    
    # 6. Weight Distribution Analysis
    ax_weight_dist = fig.add_subplot(gs[1, 2])
    optimal_weights_pct = optimal_portfolio['weights'] * 100
    ax_weight_dist.barh(assets, optimal_weights_pct, color=colors)
    ax_weight_dist.set_xlabel('Weight (%)')
    ax_weight_dist.set_title('Portfolio Weight\nDistribution')
    ax_weight_dist.grid(True, alpha=0.3)
    
    # 7. Efficient Frontier Zoom (bottom left)
    ax_ef_zoom = fig.add_subplot(gs[2, 0])
    # Focus on efficient portfolios (high Sharpe ratios)
    efficient_mask = portfolio_results['sharpe_ratio'] > np.percentile(portfolio_results['sharpe_ratio'], 90)
    ax_ef_zoom.scatter(portfolio_results['volatility'][efficient_mask], 
                      portfolio_results['returns'][efficient_mask],
                      c=portfolio_results['sharpe_ratio'][efficient_mask], 
                      cmap='viridis', alpha=0.8, s=30)
    ax_ef_zoom.scatter(optimal_portfolio['volatility'], optimal_portfolio['return'], 
                      marker='*', color='red', s=200)
    ax_ef_zoom.set_xlabel('Volatility')
    ax_ef_zoom.set_ylabel('Return')
    ax_ef_zoom.set_title('Efficient Frontier\n(Top 10% Sharpe Ratios)')
    ax_ef_zoom.grid(True, alpha=0.3)
    
    # 8. Statistics Summary Table
    ax_stats = fig.add_subplot(gs[2, 1:])
    ax_stats.axis('off')
    
    # Create summary statistics
    stats_data = [
        ['Metric', 'Value', 'Interpretation'],
        ['Expected Return', f"{optimal_portfolio['return']:.2%}", 'Annual expected return'],
        ['Volatility', f"{optimal_portfolio['volatility']:.2%}", 'Annual risk (std deviation)'],
        ['Sharpe Ratio', f"{optimal_portfolio['sharpe_ratio']:.3f}", 'Risk-adjusted return'],
        ['Max Weight', f"{max(optimal_portfolio['weights'])*100:.1f}%", 'Largest single allocation'],
        ['Min Weight', f"{min(optimal_portfolio['weights'])*100:.1f}%", 'Smallest allocation'],
        ['Diversification', f"{len([w for w in optimal_portfolio['weights'] if w > 0.05])}", 'Assets with >5% allocation']
    ]
    
    table = ax_stats.table(cellText=stats_data[1:], colLabels=stats_data[0],
                          cellLoc='center', loc='center', 
                          colWidths=[0.3, 0.2, 0.5])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style the table
    for i in range(len(stats_data)):
        for j in range(len(stats_data[0])):
            cell = table[(i, j)]
            if i == 0:  # Header row
                cell.set_facecolor('#4472C4')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#F2F2F2' if i % 2 == 0 else 'white')
    
    plt.suptitle('Comprehensive Portfolio Analysis Dashboard', fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.subplots_adjust(top=0.94)
    plt.show()

# Create comprehensive visualization
plot_comprehensive_analysis(portfolio_results, optimal_portfolio, ASSETS, 
                           expected_returns, cov_matrix, correlation_matrix)

## Step 6: VaR and CVaR with Advanced Methods

**Teaching Notes:**
- Compare different VaR calculation methods (historical, parametric, Monte Carlo)
- Explain the assumptions behind each approach
- Discuss the advantages and limitations of each method
- Emphasize the importance of backtesting VaR models

In [ ]:
def calculate_comprehensive_var_cvar(weights, returns_df, confidence_level=0.95, 
                                   time_horizon=1, num_simulations=10000):
    """
    Calculate VaR and CVaR using multiple methods for comparison.
    
    INSTRUCTOR NOTE: This function demonstrates three common approaches to VaR calculation:
    1. Historical Simulation (non-parametric)
    2. Parametric (assumes normal distribution)
    3. Monte Carlo (simulation-based)
    """
    print(f"🎯 Comprehensive VaR/CVaR Analysis")
    print(f"=" * 50)
    print(f"Confidence Level: {confidence_level:.1%}")
    print(f"Time Horizon: {time_horizon} day(s)")
    
    # Calculate portfolio returns
    portfolio_returns = (returns_df * weights).sum(axis=1)
    
    # Method 1: Historical Simulation
    print(f"\n📊 Method 1: Historical Simulation")
    historical_returns = portfolio_returns * np.sqrt(time_horizon)  # Scale for time horizon
    alpha = 1 - confidence_level
    var_historical = np.percentile(historical_returns, alpha * 100)
    cvar_historical = historical_returns[historical_returns <= var_historical].mean()
    
    print(f"  Historical VaR: {var_historical:.2%}")
    print(f"  Historical CVaR: {cvar_historical:.2%}")
    
    # Method 2: Parametric (Normal Distribution)
    print(f"\n📐 Method 2: Parametric (Normal)")
    mean_return = portfolio_returns.mean() * time_horizon
    std_return = portfolio_returns.std() * np.sqrt(time_horizon)
    var_parametric = norm.ppf(alpha, mean_return, std_return)
    # For normal distribution, CVaR has analytical formula
    cvar_parametric = mean_return - std_return * norm.pdf(norm.ppf(alpha)) / alpha
    
    print(f"  Parametric VaR: {var_parametric:.2%}")
    print(f"  Parametric CVaR: {cvar_parametric:.2%}")
    
    # Method 3: Monte Carlo Simulation
    print(f"\n🎲 Method 3: Monte Carlo Simulation")
    np.random.seed(SEED)
    simulated_returns = np.random.normal(mean_return, std_return, num_simulations)
    var_monte_carlo = np.percentile(simulated_returns, alpha * 100)
    cvar_monte_carlo = simulated_returns[simulated_returns <= var_monte_carlo].mean()
    
    print(f"  Monte Carlo VaR: {var_monte_carlo:.2%}")
    print(f"  Monte Carlo CVaR: {cvar_monte_carlo:.2%}")
    
    # INSTRUCTOR ONLY: Method comparison and analysis
    print(f"\n" + "="*50)
    print(f"🎓 INSTRUCTOR ANALYSIS: Method Comparison")
    print(f"="*50)
    
    methods_comparison = pd.DataFrame({
        'Method': ['Historical', 'Parametric', 'Monte Carlo'],
        'VaR': [var_historical, var_parametric, var_monte_carlo],
        'CVaR': [cvar_historical, cvar_parametric, cvar_monte_carlo]
    })
    methods_comparison['VaR %'] = methods_comparison['VaR'] * 100
    methods_comparison['CVaR %'] = methods_comparison['CVaR'] * 100
    
    print("VaR/CVaR Comparison:")
    print(methods_comparison[['Method', 'VaR %', 'CVaR %']].round(2).to_string(index=False))
    
    # Calculate differences
    var_range = methods_comparison['VaR'].max() - methods_comparison['VaR'].min()
    cvar_range = methods_comparison['CVaR'].max() - methods_comparison['CVaR'].min()
    
    print(f"\nMethod Sensitivity:")
    print(f"VaR range: {var_range:.2%} ({var_range/abs(methods_comparison['VaR'].mean())*100:.1f}% relative)")
    print(f"CVaR range: {cvar_range:.2%} ({cvar_range/abs(methods_comparison['CVaR'].mean())*100:.1f}% relative)")
    
    # Interpretation
    print(f"\n🔍 Method Interpretation:")
    print(f"Historical: Uses actual past data - good for capturing real market behavior")
    print(f"Parametric: Assumes normality - fast but may underestimate tail risk")
    print(f"Monte Carlo: Flexible - can incorporate complex return distributions")
    
    return {
        'historical': {'var': var_historical, 'cvar': cvar_historical},
        'parametric': {'var': var_parametric, 'cvar': cvar_parametric},
        'monte_carlo': {'var': var_monte_carlo, 'cvar': cvar_monte_carlo},
        'simulated_returns': simulated_returns,
        'historical_returns': historical_returns,
        'comparison_df': methods_comparison
    }

def plot_var_cvar_comprehensive(var_results, confidence_level):
    """
    Create comprehensive VaR/CVaR visualizations comparing different methods.
    
    INSTRUCTOR NOTE: This visualization helps students understand the differences
    between VaR calculation methods and their practical implications.
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Distribution comparison with VaR lines
    ax1.hist(var_results['historical_returns'], bins=50, alpha=0.6, 
            label='Historical Returns', density=True, color='skyblue')
    ax1.hist(var_results['simulated_returns'], bins=50, alpha=0.6, 
            label='Simulated Returns', density=True, color='lightcoral')
    
    # Add VaR lines
    colors = ['blue', 'green', 'red']
    methods = ['Historical', 'Parametric', 'Monte Carlo']
    vars = [var_results['historical']['var'], 
            var_results['parametric']['var'], 
            var_results['monte_carlo']['var']]
    
    for i, (method, var_val, color) in enumerate(zip(methods, vars, colors)):
        ax1.axvline(var_val, color=color, linestyle='--', linewidth=2, 
                   label=f'{method} VaR')
    
    ax1.set_xlabel('Portfolio Returns')
    ax1.set_ylabel('Density')
    ax1.set_title(f'Return Distributions with VaR ({confidence_level:.0%})')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. VaR Method Comparison
    methods = var_results['comparison_df']['Method']
    var_values = var_results['comparison_df']['VaR %']
    cvar_values = var_results['comparison_df']['CVaR %']
    
    x = np.arange(len(methods))
    width = 0.35
    
    bars1 = ax2.bar(x - width/2, var_values, width, label='VaR', alpha=0.8, color='lightblue')
    bars2 = ax2.bar(x + width/2, cvar_values, width, label='CVaR', alpha=0.8, color='lightcoral')
    
    ax2.set_xlabel('Method')
    ax2.set_ylabel('Value (%)')
    ax2.set_title('VaR vs CVaR by Method')
    ax2.set_xticks(x)
    ax2.set_xticklabels(methods)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar in bars1 + bars2:
        height = bar.get_height()
        ax2.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)
    
    # 3. Tail Risk Analysis
    tail_threshold = var_results['monte_carlo']['var']
    tail_returns = var_results['simulated_returns'][var_results['simulated_returns'] <= tail_threshold]
    
    ax3.hist(var_results['simulated_returns'], bins=100, alpha=0.6, color='lightblue', 
            density=True, label='All Returns')
    ax3.hist(tail_returns, bins=30, alpha=0.8, color='red', density=True, 
            label=f'Tail Risk ({100-confidence_level*100:.0f}%)')
    ax3.axvline(tail_threshold, color='red', linestyle='--', linewidth=2, label='VaR Threshold')
    ax3.axvline(var_results['monte_carlo']['cvar'], color='darkred', linestyle='--', 
               linewidth=2, label='CVaR (Expected Tail Loss)')
    
    ax3.set_xlabel('Portfolio Returns')
    ax3.set_ylabel('Density')
    ax3.set_title('Tail Risk Analysis')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Cumulative Distribution with Risk Measures
    sorted_returns = np.sort(var_results['simulated_returns'])
    probabilities = np.arange(1, len(sorted_returns) + 1) / len(sorted_returns)
    
    ax4.plot(sorted_returns, probabilities, color='blue', linewidth=2, label='Cumulative Distribution')
    ax4.axvline(var_results['monte_carlo']['var'], color='red', linestyle='--', 
               linewidth=2, label=f'VaR ({confidence_level:.0%})')
    ax4.axhline(1 - confidence_level, color='red', linestyle=':', alpha=0.7, 
               label=f'{confidence_level:.0%} Confidence Level')
    
    # Shade the tail area
    tail_mask = sorted_returns <= var_results['monte_carlo']['var']
    ax4.fill_between(sorted_returns[tail_mask], 0, probabilities[tail_mask], 
                    alpha=0.3, color='red', label='Tail Risk Area')
    
    ax4.set_xlabel('Portfolio Returns')
    ax4.set_ylabel('Cumulative Probability')
    ax4.set_title('Cumulative Distribution with Risk Measures')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Calculate comprehensive VaR/CVaR
var_results = calculate_comprehensive_var_cvar(
    optimal_portfolio['weights'], returns_df, CONFIDENCE_LEVEL, 
    time_horizon=1, num_simulations=10000 if not SAMPLE_MODE else 5000
)

# Create comprehensive visualization
plot_var_cvar_comprehensive(var_results, CONFIDENCE_LEVEL)

## INSTRUCTOR SOLUTIONS: Exercise Implementations

**Teaching Notes:**
- These solutions demonstrate advanced portfolio optimization techniques
- Show students how to implement constraints and different objective functions
- Explain the trade-offs between different optimization approaches
- Discuss practical considerations for real-world implementation

In [ ]:
# INSTRUCTOR SOLUTION 1: Risk-Free Asset Integration
print("🎓 INSTRUCTOR SOLUTION 1: Risk-Free Asset Integration")
print("=" * 60)

def add_risk_free_asset(expected_returns, cov_matrix, risk_free_rate):
    """
    Add a risk-free asset to the portfolio universe.
    
    INSTRUCTOR NOTE: Adding a risk-free asset changes the efficient frontier from
    a curve to a straight line (Capital Allocation Line) for portfolios that include
    the risk-free asset.
    """
    # Create new expected returns including risk-free asset
    rf_returns = expected_returns.copy()
    rf_returns['RF'] = risk_free_rate
    
    # Create new covariance matrix (risk-free asset has zero covariance with everything)
    rf_cov = cov_matrix.copy()
    rf_cov['RF'] = 0
    rf_cov.loc['RF'] = 0
    rf_cov.loc['RF', 'RF'] = 0  # Risk-free asset has zero variance
    
    return rf_returns, rf_cov

def optimize_with_risk_free_asset(expected_returns, cov_matrix, risk_free_rate):
    """Optimize portfolio including risk-free asset."""
    rf_returns, rf_cov = add_risk_free_asset(expected_returns, cov_matrix, risk_free_rate)
    n_assets = len(rf_returns)
    
    def negative_sharpe_ratio(weights):
        portfolio_return = np.sum(rf_returns * weights)
        portfolio_variance = np.dot(weights.T, np.dot(rf_cov, weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        # Avoid division by zero
        if portfolio_volatility == 0:
            return -np.inf
        return -(portfolio_return - risk_free_rate) / portfolio_volatility
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n_assets))
    initial_guess = np.array([1/n_assets] * n_assets)
    
    result = minimize(negative_sharpe_ratio, initial_guess, 
                     method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        return {
            'weights': result.x,
            'assets': list(rf_returns.index),
            'returns': rf_returns,
            'success': True
        }
    return {'success': False}

# Implement risk-free asset solution
rf_solution = optimize_with_risk_free_asset(expected_returns, cov_matrix, RISK_FREE_RATE)

if rf_solution['success']:
    print("Portfolio with Risk-Free Asset:")
    for i, (asset, weight) in enumerate(zip(rf_solution['assets'], rf_solution['weights'])):
        if weight > 0.001:  # Only show significant weights
            print(f"  {asset}: {weight*100:.1f}%")
    
    rf_weight = rf_solution['weights'][-1]  # Risk-free asset is last
    print(f"\nRisk-Free Asset Allocation: {rf_weight*100:.1f}%")
    print(f"Risky Asset Allocation: {(1-rf_weight)*100:.1f}%")
else:
    print("❌ Risk-free asset optimization failed")

print("\n" + "="*60)
print("🎓 INSTRUCTOR SOLUTION 2: Constrained Optimization")
print("="*60)

def optimize_with_constraints(expected_returns, cov_matrix, risk_free_rate, 
                            min_weight=0.05, max_weight=0.40):
    """
    Optimize portfolio with realistic constraints.
    
    INSTRUCTOR NOTE: Real-world portfolios often have constraints:
    - No short selling (min_weight >= 0)
    - Diversification requirements (max_weight <= threshold)
    - Minimum allocations (min_weight >= threshold)
    """
    n_assets = len(expected_returns)
    
    def negative_sharpe_ratio(weights):
        portfolio_return = np.sum(expected_returns * weights)
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        return -(portfolio_return - risk_free_rate) / portfolio_volatility
    
    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Weights sum to 1
    ]
    
    # Bounds: minimum and maximum weights
    bounds = tuple((min_weight, max_weight) for _ in range(n_assets))
    
    # Initial guess: equal weights within bounds
    initial_weight = 1 / n_assets
    if initial_weight < min_weight:
        initial_weight = min_weight
    elif initial_weight > max_weight:
        initial_weight = max_weight
    
    initial_guess = np.array([initial_weight] * n_assets)
    initial_guess = initial_guess / np.sum(initial_guess)  # Normalize
    
    result = minimize(negative_sharpe_ratio, initial_guess, 
                     method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        optimal_weights = result.x
        portfolio_return = np.sum(expected_returns * optimal_weights)
        portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        
        return {
            'weights': optimal_weights,
            'return': portfolio_return,
            'volatility': portfolio_volatility,
            'sharpe_ratio': sharpe_ratio,
            'success': True
        }
    return {'success': False}

# Implement constrained optimization
constrained_solution = optimize_with_constraints(expected_returns, cov_matrix, RISK_FREE_RATE)

if constrained_solution['success']:
    print("Constrained Portfolio (5%-40% per asset):")
    constrained_df = pd.DataFrame({
        'Asset': ASSETS,
        'Weight': constrained_solution['weights'],
        'Weight %': constrained_solution['weights'] * 100
    }).sort_values('Weight %', ascending=False)
    
    for _, row in constrained_df.iterrows():
        print(f"  {row['Asset']}: {row['Weight %']:.1f}%")
    
    print(f"\nConstrained Portfolio Metrics:")
    print(f"  Expected Return: {constrained_solution['return']:.2%}")
    print(f"  Volatility: {constrained_solution['volatility']:.2%}")
    print(f"  Sharpe Ratio: {constrained_solution['sharpe_ratio']:.3f}")
    
    # Compare with unconstrained
    print(f"\nComparison with Unconstrained:")
    print(f"  Sharpe Ratio Difference: {constrained_solution['sharpe_ratio'] - optimal_portfolio['sharpe_ratio']:.3f}")
    if constrained_solution['sharpe_ratio'] < optimal_portfolio['sharpe_ratio']:
        loss = (optimal_portfolio['sharpe_ratio'] - constrained_solution['sharpe_ratio']) / optimal_portfolio['sharpe_ratio']
        print(f"  Performance Loss: {loss*100:.1f}%")
    
else:
    print("❌ Constrained optimization failed")

print("\n" + "="*60)
print("🎓 INSTRUCTOR SOLUTION 3: Multiple Objective Functions")
print("="*60)

def optimize_multiple_objectives(expected_returns, cov_matrix, risk_free_rate):
    """
    Demonstrate different optimization objectives.
    
    INSTRUCTOR NOTE: Different objectives lead to different optimal portfolios:
    - Max Sharpe: Best risk-adjusted return
    - Min Variance: Lowest risk
    - Max Return: Highest return (subject to risk constraint)
    """
    n_assets = len(expected_returns)
    
    # 1. Maximum Sharpe Ratio (already done)
    max_sharpe = optimal_portfolio
    
    # 2. Minimum Variance Portfolio
    def portfolio_variance(weights):
        return np.dot(weights.T, np.dot(cov_matrix, weights))
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n_assets))
    initial_guess = np.array([1/n_assets] * n_assets)
    
    min_var_result = minimize(portfolio_variance, initial_guess, 
                             method='SLSQP', bounds=bounds, constraints=constraints)
    
    if min_var_result.success:
        min_var_weights = min_var_result.x
        min_var_return = np.sum(expected_returns * min_var_weights)
        min_var_volatility = np.sqrt(portfolio_variance(min_var_weights))
        min_var_sharpe = (min_var_return - risk_free_rate) / min_var_volatility
        
        min_var_portfolio = {
            'weights': min_var_weights,
            'return': min_var_return,
            'volatility': min_var_volatility,
            'sharpe_ratio': min_var_sharpe
        }
    
    # 3. Maximum Return Portfolio (equal weights for comparison)
    equal_weights = np.array([1/n_assets] * n_assets)
    equal_return = np.sum(expected_returns * equal_weights)
    equal_variance = np.dot(equal_weights.T, np.dot(cov_matrix, equal_weights))
    equal_volatility = np.sqrt(equal_variance)
    equal_sharpe = (equal_return - risk_free_rate) / equal_volatility
    
    equal_portfolio = {
        'weights': equal_weights,
        'return': equal_return,
        'volatility': equal_volatility,
        'sharpe_ratio': equal_sharpe
    }
    
    # Create comparison
    objectives_df = pd.DataFrame({
        'Objective': ['Max Sharpe', 'Min Variance', 'Equal Weights'],
        'Return': [max_sharpe['return'], min_var_portfolio['return'], equal_portfolio['return']],
        'Volatility': [max_sharpe['volatility'], min_var_portfolio['volatility'], equal_portfolio['volatility']],
        'Sharpe': [max_sharpe['sharpe_ratio'], min_var_portfolio['sharpe_ratio'], equal_portfolio['sharpe_ratio']]
    })
    
    print("Portfolio Comparison by Objective:")
    print(objectives_df.round(4).to_string(index=False))
    
    return {
        'max_sharpe': max_sharpe,
        'min_variance': min_var_portfolio,
        'equal_weights': equal_portfolio,
        'comparison': objectives_df
    }

# Implement multiple objectives comparison
multi_obj_results = optimize_multiple_objectives(expected_returns, cov_matrix, RISK_FREE_RATE)

print(f"\n🎯 Key Teaching Points:")
print(f"1. Max Sharpe provides best risk-adjusted returns")
print(f"2. Min Variance provides lowest risk but may sacrifice returns")
print(f"3. Equal weights is simple but usually sub-optimal")
print(f"4. Choice depends on investor risk tolerance and objectives")

## Summary and Advanced Topics

**Teaching Summary:**
This notebook demonstrates the complete portfolio optimization workflow using Modern Portfolio Theory. Students have learned to:

1. **Load and process financial data** with robust error handling
2. **Calculate portfolio statistics** including returns, volatility, and correlations  
3. **Implement Monte Carlo simulation** for portfolio exploration
4. **Find optimal portfolios** using both simulation and analytical methods
5. **Calculate risk metrics** (VaR/CVaR) using multiple approaches
6. **Create comprehensive visualizations** for analysis and presentation
7. **Apply practical constraints** for real-world portfolio management

**Advanced Extensions Completed:**
- ✅ Risk-free asset integration and Capital Allocation Line
- ✅ Constrained optimization with realistic bounds
- ✅ Multiple objective function comparison
- ✅ Comprehensive risk measurement methods
- ✅ Advanced visualization dashboard

**Next Steps for Students:**
- Implement dynamic rebalancing strategies
- Add transaction costs and market impact
- Explore alternative risk measures (drawdown, semi-variance)
- Study factor models and risk attribution
- Learn about Black-Litterman model improvements

---

**🎓 Instructor Final Notes:**
- Emphasize MPT assumptions and limitations in practice
- Discuss estimation risk and parameter uncertainty
- Connect to broader portfolio management concepts
- Encourage critical thinking about model limitations
- Prepare students for advanced portfolio theory topics

In [ ]:
# Final execution summary and teaching assessment
import time
print("✅ INSTRUCTOR NOTEBOOK COMPLETED")
print("=" * 60)

print(f"📊 Comprehensive Analysis Summary:")
print(f"  - Complete portfolio optimization implementation")
print(f"  - Multiple VaR/CVaR calculation methods")
print(f"  - Advanced constraint handling")
print(f"  - Comprehensive visualization dashboard")
print(f"  - All exercise solutions provided")

print(f"\n🎯 Student Learning Outcomes Achieved:")
print(f"  ✅ Modern Portfolio Theory implementation")
print(f"  ✅ Monte Carlo simulation techniques")
print(f"  ✅ Risk measurement and management")
print(f"  ✅ Optimization with practical constraints")
print(f"  ✅ Professional-quality analysis and visualization")

print(f"\n📚 Teaching Effectiveness:")
print(f"  - Theory connected to practical implementation")
print(f"  - Multiple solution approaches demonstrated")
print(f"  - Real-world constraints and considerations")
print(f"  - Comprehensive error handling and robustness")

print(f"\n⏱️ Instructor notebook execution completed!")
print(f"   Ready for classroom deployment and student guidance")